In [1]:
import pandas as pd
import os
import requests as req
import json
import numpy as np
import datetime
from matplotlib import style
import matplotlib
%matplotlib inline
style.use('ggplot')

# This is to import custom-made modules

# This can be removed after making these modules a real library
import os, sys
#lib_path = os.path.abspath(os.path.join('..', 'src')) # relative path of the source code in Box Folder 
#sys.path.append(lib_path)

In [10]:
data = pd.read_csv(os.path.join(".","Units-1311.csv")) 

In [31]:
data['opponent'].unique()

array(['Z', 'T', 'P'], dtype=object)

In [33]:
#data2 = pd.read_csv(os.path.join("..","lossreplay.csv")) 

data.replace({'opponent' : { 'T' : 1, 'Z' : 2, 'P' : 3 }}, inplace =True)
data.head()

,winner,opponent,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,Ghost,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
#data.columns

In [34]:
X = data.iloc[:,1:]
y  = data['winner']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [36]:
from sklearn.neighbors import KNeighborsClassifier

In [37]:
knn = KNeighborsClassifier(n_neighbors = 10)

In [38]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [39]:
knn.score(X_test, y_test)

0.8094527404206898

In [25]:
X_test.head()

,seconds,BarracksTechLab,Hellion,MULE,VikingFighter,Barracks,SiegeTankSieged,Ghost,SupplyDepot,Factory,...,SupplyDepotLowered,MissileTurret,Starport,Nuke,StarportTechLab,StarportReactor,SensorTower,Marauder,BarracksReactor,PointDefenseDrone
467598,1184,0,4,0,0,1,16,0,13,9,...,8,11,5,0,1,1,2,0,0,0
27421,1568,3,0,6,0,8,60,0,12,4,...,22,0,2,0,0,1,1,0,3,0
261723,1609,0,0,0,0,2,18,2,1,1,...,26,18,4,1,2,1,1,0,1,0
475093,484,0,0,0,0,1,2,0,4,0,...,0,1,1,0,0,0,1,2,0,0
300720,1491,0,0,7,0,1,62,0,13,5,...,5,2,4,0,2,2,0,0,1,0


In [26]:
y_test.head()

467598    1
27421     0
261723    1
475093    0
300720    0
Name: winner, dtype: int64

In [28]:
X_test.shape

(121277, 53)

In [77]:
forMatt = data.iloc[2:50,1:]

In [93]:
predictions = knn.predict_proba(forMatt)

In [80]:
predictions[0][1]

0.2

In [65]:
predictions[2][1]

0.5

In [94]:
len(predictions)

48

In [95]:
mylist =[]
for i in xrange(len(predictions)):
    mylist.append(predictions[i][1])

In [96]:
forMatt['prediction'] = mylist

In [99]:
toplot = pd.DataFrame()
toplot['seconds'] = forMatt['seconds']
toplot['prediction'] = forMatt['prediction']
toplot.head()

,seconds,prediction
2,0,0.2
3,0,0.6
4,0,0.5
5,0,0.3
6,0,0.4


In [100]:
toplot.to_csv('forMatt.csv')